# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv

# Import API key
from api_keys import geoapify_key

C:\Users\jhaug\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dunedin,-45.8742,170.5036,12.74,86,12,5.92,NZ,1712037215
1,1,tiksi,71.6872,128.8694,-11.70,88,98,8.93,RU,1712037322
2,2,pont rouge,48.8833,-72.0825,-1.69,78,12,2.96,CA,1712037322
3,3,puerto baquerizo moreno,-0.9000,-89.6000,24.25,81,65,2.43,EC,1712037322
4,4,puerto ayora,-0.7393,-90.3518,23.79,97,80,0.89,EC,1712037322


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
     geo = True,
     tiles = "OSM",
     width = 1000,
     height = 500,
     color = "City",  # generates the side legend
     size = "Humidity"
)

# Display the map plot
# Note to grader: Must restart kernel to view more than one time; map is only good for one iteration....
map_plot_1

# Note: Leaving x- and y-axes as Longitude and Latitude, respectively, for presentation; not using x and y

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Looking for a place that is between 32 and 77 degrees Farenheit, humidity less than 50% 

narrow_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] > 0) & (city_data_df["Max Temp"] < 25) & (city_data_df["Humidity"] < 50)]
narrow_city_data_df = narrow_city_data_df.dropna(axis = 1)
narrow_city_data_df.reset_index(inplace=True)
#del narrow_city_data_df['index']
narrow_city_data_df.head()

# Drop any rows with null values
narrow_city_data_df.dropna(axis = 0) 

# Display sample data
narrow_city_data_df.head()

,index,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,23,23,college,64.8569,-147.8028,3.14,44,75,4.12,1712037326
1,37,37,badger,64.8000,-147.5333,3.10,44,0,4.12,1712037329
2,39,39,swift current,50.2834,-107.8014,5.60,42,0,1.03,1712037063
3,50,50,khabarovsk,48.4808,135.0928,7.04,43,20,11.18,1712037088
4,61,61,kushiro,42.9750,144.3747,7.68,49,40,2.57,1712037335


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

C:\Users\jhaug\AppData\Local\Temp\ipykernel_39664\1708204268.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
0,dunedin,NZ,-45.8742,170.5036,86,
1,tiksi,RU,71.6872,128.8694,88,
2,pont rouge,CA,48.8833,-72.0825,78,
3,puerto baquerizo moreno,EC,-0.9000,-89.6000,81,
4,puerto ayora,EC,-0.7393,-90.3518,97,
...,...,...,...,...,...,...
533,vammala,FI,61.3333,22.9000,89,
534,susanville,US,40.4163,-120.6530,79,
535,zaysan,KZ,47.4667,84.8667,55,
536,rafsanjan,IR,30.4067,55.9939,39,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
limit = 1  # finding the first hotel 
params = {
    "limit":limit,
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    params["categories"] = f'accommodation.hotel'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']}: nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data

hotel_df.head() # Displays the first hotel

#  Note:  The first hotel will be the first hotel in the list being identified with "nearest hotel"


Starting hotel search
dunedin: nearest hotel: Scenic Hotel Dunedin City
tiksi: nearest hotel: Арктика
pont rouge: nearest hotel: No hotel found
puerto baquerizo moreno: nearest hotel: Dory's House
puerto ayora: nearest hotel: Hostal La Mirada De Solitario George
whitehorse: nearest hotel: Town & Mountain Hotel
papatowai: nearest hotel: No hotel found
waitangi: nearest hotel: Hotel Chathams
port-aux-francais: nearest hotel: Keravel
ushuaia: nearest hotel: Apart Hotel Aires del Beagle
port elizabeth: nearest hotel: Waterford Hotel
kodiak: nearest hotel: No hotel found
west island: nearest hotel: Cocos Village Bungalows
avarua: nearest hotel: Paradise Inn
punta arenas: nearest hotel: Hotel Lacolet
castillos: nearest hotel: A mi gente
santarem: nearest hotel: Prático Hotel
vilyuchinsk: nearest hotel: Победа
south williamsport: nearest hotel: Hampton Inn & Suites
beterou: nearest hotel: No hotel found
jacobina: nearest hotel: Hotel Jovem Jota
mount isa: nearest hotel: Ibis Styles
haiku-pauw

,City,Country,Lat,Lng,Humidity,Hotel Name
0,dunedin,NZ,-45.8742,170.5036,86,Scenic Hotel Dunedin City


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# Configure the map plot
map_plot_2 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
     geo = True,
     tiles = "OSM",
     width = 1000,
     height = 500,
     color= "City",  # generates the side legend
     size = "Humidity",   # maintaining size of points/markers
     hover_cols = ["Hotel Name","Country"]
)

# Display the map
map_plot_2

# Note: Leaving x- and y-axes as Longitude and Latitude, respectively, for presentation; not using x and y

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)